# Multi-Format Document Search Index Pipeline

This notebook creates a comprehensive document search index from multiple file types, **always starting fresh** for testing and development. Every run will delete and recreate the index to ensure the schema is always up to date.

## Supported File Types

- **📝 Markdown files (.md)** - With smart title extraction from headings
- **📄 Text files (.txt)** - Plain text processing  
- **📊 CSV files (.csv)** - Intelligent table parsing with special handling for statistics files
- **📑 PDF files (.pdf)** - Text extraction from all pages
- **📈 Excel files (.xlsx/.xls)** - Multi-sheet processing with column structure
- **⚙️ JSON files (.json)** - Structured data formatting

## Pipeline Flow

1. **🗑️ Delete existing index** - Ensures clean state
2. **🏗️ Create fresh index** - With latest schema including metadata fields
3. **📁 Process documents** - Smart content extraction and chunking
4. **📤 Upload to index** - Batch processing with error handling
5. **📊 Show statistics** - Summary of indexed content

## Requirements

You'll need credentials for the following Azure services:

- **Azure Search Service** - For the vector search index
- **Azure OpenAI Service** - For text embeddings

Find the names and keys in the Azure Portal and add them to a `.env` file in the root of this repository. The `.env` file is not checked in to source control. You can use the [`.env.sample`](../../.env.sample) file as a template.

## Fresh Index Benefits

This "always fresh" approach is perfect for:
- **Testing new features** - No schema conflicts
- **Iterative development** - Clean state every run  
- **Content updates** - All changes are captured
- **Schema evolution** - New fields work immediately

In [16]:
import os
import pandas as pd
from azure.identity import DefaultAzureCredential
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    HnswParameters,
    HnswAlgorithmConfiguration,
    SemanticPrioritizedFields,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticSearch,
    SemanticConfiguration,
    SemanticField,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters
)
from typing import List, Dict
from openai import AzureOpenAI
from dotenv import load_dotenv
from pathlib import Path
import csv
import json
import mimetypes

# Additional imports for document processing
try:
    import PyPDF2
    PDF_AVAILABLE = True
except ImportError:
    PDF_AVAILABLE = False
    print("Warning: PyPDF2 not available. Install with: pip install PyPDF2")

try:
    import openpyxl
    EXCEL_AVAILABLE = True
except ImportError:
    EXCEL_AVAILABLE = False
    print("Warning: openpyxl not available. Install with: pip install openpyxl")

load_dotenv()

True

In [17]:
def delete_index(search_index_client: SearchIndexClient, search_index: str):
    print(f"deleting index {search_index}")
    search_index_client.delete_index(search_index)

In [ ]:
# Document processing functions for different file types with proper encoding handling

def detect_encoding(file_path: Path) -> str:
    """Detect the encoding of a text file, with preference for UTF-8 and German-specific encodings"""
    try:
        # Try common encodings for German text
        encodings_to_try = ['utf-8', 'utf-8-sig', 'iso-8859-1', 'cp1252', 'latin1']
        
        for encoding in encodings_to_try:
            try:
                with open(file_path, 'r', encoding=encoding) as f:
                    # Read a sample to test encoding
                    sample = f.read(1024)
                    # Check if it contains common German characters properly
                    if any(char in sample for char in ['ä', 'ö', 'ü', 'ß', 'Ä', 'Ö', 'Ü']):
                        return encoding
                    elif encoding == 'utf-8' and not any(char in sample for char in ['Ã']):
                        # UTF-8 without encoding artifacts
                        return encoding
            except (UnicodeDecodeError, UnicodeError):
                continue
        
        # Default to utf-8 if nothing works
        return 'utf-8'
    except Exception:
        return 'utf-8'

def clean_german_text(text: str) -> str:
    """Clean up common German encoding issues"""
    # Fix common UTF-8 encoding artifacts for German umlauts
    replacements = {
        'Ã¤': 'ä',
        'Ã¶': 'ö', 
        'Ã¼': 'ü',
        'Ã': 'ß',
        'Ã„': 'Ä',
        'Ã–': 'Ö',
        'Ãœ': 'Ü',
        'â€œ': '"',
        'â€': '"',
        'â€™': "'",
        'â€"': '–',
        'â€"': '—'
    }
    
    for bad, good in replacements.items():
        text = text.replace(bad, good)
    
    return text

def process_csv_file(file_path: Path) -> str:
    """Process CSV files with intelligent content extraction and proper encoding, with special handling for German statistical data"""
    try:
        # Detect encoding for better German text handling
        encoding = detect_encoding(file_path)
        
        # Special handling for German BKA statistics files
        if "statistic" in file_path.name.lower() or "bka" in file_path.name.lower():
            return process_german_statistics_csv_integrated(file_path, encoding)
        
        # Try to read with pandas using detected encoding
        df = pd.read_csv(file_path, encoding=encoding)
        
        # Check if this looks like a statistics table (like your example)
        if df.columns[0].startswith('Unnamed'):
            # This looks like a messy CSV with metadata, try to clean it
            df = pd.read_csv(file_path, encoding=encoding, header=None)
            
            # Extract meaningful content
            content_parts = []
            
            # Look for title or description rows
            for idx, row in df.iterrows():
                row_text = ' '.join([str(val) for val in row.values if pd.notna(val) and str(val).strip()])
                if len(row_text.strip()) > 10:  # Skip empty or very short rows
                    content_parts.append(row_text.strip())
            
            # Join the meaningful parts
            content = '\n'.join(content_parts)
            
        else:
            # Regular CSV - convert to readable format
            content_parts = [f"CSV Data from {file_path.name}"]
            content_parts.append(f"Columns: {', '.join(df.columns)}")
            content_parts.append("")
            
            # Add first few rows as examples
            for idx, row in df.head(10).iterrows():
                row_data = []
                for col in df.columns:
                    if pd.notna(row[col]):
                        row_data.append(f"{col}: {row[col]}")
                if row_data:
                    content_parts.append(" | ".join(row_data))
            
            if len(df) > 10:
                content_parts.append(f"... and {len(df) - 10} more rows")
            
            content = '\n'.join(content_parts)
        
        # Clean up any encoding artifacts
        content = clean_german_text(content)
        return content
        
    except Exception as e:
        return f"Error processing CSV file {file_path.name}: {str(e)}"

def process_german_statistics_csv_integrated(file_path: Path, encoding: str = 'utf-8') -> str:
    """
    Specialized processor for German statistics CSV files like BKA crime statistics
    Integrated into the normal multi-file processing workflow
    """
    try:
        print(f"   📊 Using specialized German statistics processing")
        
        # Read the raw file with proper encoding
        with open(file_path, 'r', encoding=encoding) as f:
            lines = f.readlines()
        
        content_parts = []
        content_parts.append(f"German Statistical Data from {file_path.name}")
        content_parts.append("")
        
        # Extract meaningful rows with German text and numbers
        meaningful_rows = []
        data_rows = []
        
        for i, line in enumerate(lines):
            line = line.strip()
            
            # Skip empty lines and pure comma lines
            if not line or line.replace(',', '').strip() == '':
                continue
                
            # Parse CSV row
            try:
                row_parts = [part.strip().strip('"') for part in line.split(',')]
                row_text = ' '.join([part for part in row_parts if part and part != 'nan'])
                
                # Check if this row contains meaningful German text (more than just column headers)
                if len(row_text) > 20:
                    # Key statistical content - look for German keywords and numbers
                    if any(keyword in row_text.lower() for keyword in [
                        'straftat', 'verdächtig', 'kriminal', 'delikt', 'deutschland', 
                        'polizei', 'ermittelt', 'ausländer', 'rohheit', 'diebstahl'
                    ]):
                        meaningful_rows.append(f"Zeile {i+1}: {row_text}")
                        
                        # If it contains large numbers, it's likely key statistical data
                        if any(num in row_text for num in ['000', '.000', ',000']):
                            content_parts.append(f"📊 WICHTIGE STATISTIK: {row_text}")
                    
                    # Data rows with specific crime categories and numbers
                    elif any(crime_type in row_text for crime_type in [
                        'Strafrechtliche Nebengesetze', 'Rohheitsdelikte', 'Diebstahlsdelikte',
                        'Vermögens- und Fälschungsdelikte', 'Gewaltkriminalität', 'Rauschgiftkriminalität'
                    ]):
                        data_rows.append(row_text)
                        
            except Exception as e:
                # If CSV parsing fails, just treat as text
                if len(line) > 20 and any(keyword in line.lower() for keyword in [
                    'straftat', 'verdächtig', 'deutschland', 'polizei', 'ausländer'
                ]):
                    meaningful_rows.append(f"Text Zeile {i+1}: {line}")
        
        # Add meaningful content to the result
        if meaningful_rows:
            content_parts.append("BESCHREIBUNG UND KONTEXT:")
            content_parts.extend(meaningful_rows[:10])  # Limit to avoid too much metadata
            content_parts.append("")
        
        if data_rows:
            content_parts.append("STATISTISCHE DATEN:")
            content_parts.extend(data_rows)
            content_parts.append("")
        
        # Add source information
        content_parts.append("QUELLE: Bundeskriminalamt (BKA)")
        content_parts.append("JAHR: 2024")
        content_parts.append("REGION: Deutschland")
        
        result = '\n'.join(content_parts)
        
        # Clean up any encoding issues
        result = clean_german_text(result)
        
        print(f"   ✅ German statistics: {len(meaningful_rows)} description rows, {len(data_rows)} data rows")
        
        return result
        
    except Exception as e:
        print(f"   ❌ Error in German statistics processing: {e}")
        return f"Error processing German statistics file {file_path.name}: {str(e)}"

def process_pdf_file(file_path: Path) -> str:
    """Process PDF files to extract text content with proper encoding"""
    if not PDF_AVAILABLE:
        return f"PDF processing not available for {file_path.name}. Install PyPDF2."
    
    try:
        content_parts = []
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            for page_num, page in enumerate(pdf_reader.pages, 1):
                text = page.extract_text()
                if text.strip():
                    # Clean German encoding issues
                    text = clean_german_text(text)
                    content_parts.append(f"Page {page_num}:\n{text.strip()}")
        
        return '\n\n'.join(content_parts)
        
    except Exception as e:
        return f"Error processing PDF file {file_path.name}: {str(e)}"

def process_excel_file(file_path: Path) -> str:
    """Process Excel files with multiple sheets and proper encoding"""
    if not EXCEL_AVAILABLE:
        return f"Excel processing not available for {file_path.name}. Install openpyxl."
    
    try:
        content_parts = [f"Excel Data from {file_path.name}"]
        
        # Read all sheets
        xl_file = pd.ExcelFile(file_path)
        
        for sheet_name in xl_file.sheet_names:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            
            content_parts.append(f"\nSheet: {sheet_name}")
            content_parts.append(f"Columns: {', '.join(df.columns)}")
            
            # Add first few rows
            for idx, row in df.head(5).iterrows():
                row_data = []
                for col in df.columns:
                    if pd.notna(row[col]):
                        row_data.append(f"{col}: {row[col]}")
                if row_data:
                    content_parts.append(" | ".join(row_data))
            
            if len(df) > 5:
                content_parts.append(f"... and {len(df) - 5} more rows")
        
        content = '\n'.join(content_parts)
        # Clean German encoding issues
        content = clean_german_text(content)
        return content
        
    except Exception as e:
        return f"Error processing Excel file {file_path.name}: {str(e)}"

def process_text_file(file_path: Path) -> str:
    """Process plain text files with proper encoding detection"""
    try:
        # Detect best encoding for the file
        encoding = detect_encoding(file_path)
        
        with open(file_path, 'r', encoding=encoding) as f:
            content = f.read()
        
        # Clean up any encoding artifacts
        content = clean_german_text(content)
        return content
        
    except Exception as e:
        # Fallback encodings
        encodings_to_try = ['utf-8', 'iso-8859-1', 'cp1252', 'latin1']
        
        for encoding in encodings_to_try:
            try:
                with open(file_path, 'r', encoding=encoding) as f:
                    content = f.read()
                content = clean_german_text(content)
                return content
            except (UnicodeDecodeError, UnicodeError):
                continue
        
        return f"Error reading text file {file_path.name}: {str(e)}"

def process_json_file(file_path: Path) -> str:
    """Process JSON files with proper encoding"""
    try:
        # Detect encoding first
        encoding = detect_encoding(file_path)
        
        with open(file_path, 'r', encoding=encoding) as f:
            data = json.load(f)
        
        # Convert JSON to readable format
        content_parts = [f"JSON Data from {file_path.name}"]
        json_content = json.dumps(data, indent=2, ensure_ascii=False)
        
        # Clean up any encoding artifacts
        json_content = clean_german_text(json_content)
        content_parts.append(json_content)
        
        return '\n'.join(content_parts)
        
    except Exception as e:
        return f"Error processing JSON file {file_path.name}: {str(e)}"

def get_file_processor(file_path: Path):
    """Get the appropriate processor function for a file type"""
    suffix = file_path.suffix.lower()
    
    processors = {
        '.md': process_text_file,
        '.txt': process_text_file,
        '.csv': process_csv_file,
        '.pdf': process_pdf_file,
        '.xlsx': process_excel_file,
        '.xls': process_excel_file,
        '.json': process_json_file,
    }
    
    return processors.get(suffix, process_text_file)

In [19]:
def create_index_definition(name: str) -> SearchIndex:
    """
    Returns an Azure Cognitive Search index with the given name.
    """
    # The fields we want to index. The "embedding" field is a vector field that will
    # be used for vector search.
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="content", type=SearchFieldDataType.String),
        SimpleField(name="filepath", type=SearchFieldDataType.String),
        SearchableField(name="title", type=SearchFieldDataType.String),
        SimpleField(name="url", type=SearchFieldDataType.String),
        # Additional metadata fields for better referencing
        SimpleField(name="originalFilename", type=SearchFieldDataType.String),
        SearchableField(name="originalTitle", type=SearchFieldDataType.String),
        SimpleField(name="documentStem", type=SearchFieldDataType.String),
        SimpleField(name="documentType", type=SearchFieldDataType.String),  # New: file type
        SimpleField(name="fileExtension", type=SearchFieldDataType.String),  # New: file extension
        SimpleField(name="chunkIndex", type=SearchFieldDataType.Int32),
        SimpleField(name="totalChunks", type=SearchFieldDataType.Int32),
        SimpleField(name="isChunked", type=SearchFieldDataType.Boolean),
        SearchField(
            name="contentVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            # Size of the vector created by the text-embedding-3-large model.
            vector_search_dimensions=3072,
            vector_search_profile_name="myHnswProfile",
        ),
        SearchField(
            name="titleVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            # Size of the vector created by the text-embedding-3-large model.
            vector_search_dimensions=3072,
            vector_search_profile_name="myHnswProfile",
        ),    
    ]

    # The "content" field should be prioritized for semantic ranking.
    semantic_config = SemanticConfiguration(
        name="default",
        prioritized_fields=SemanticPrioritizedFields(
            title_field=SemanticField(field_name="title"),
            keywords_fields=[],
            content_fields=[SemanticField(field_name="content")],
        ),
    )

    # For vector search, we want to use the HNSW (Hierarchical Navigable Small World)
    # algorithm (a type of approximate nearest neighbor search algorithm) with cosine
    # distance.
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw",
                kind=VectorSearchAlgorithmKind.HNSW,
                parameters=HnswParameters(
                    m=4,
                    ef_construction=400,
                    ef_search=500,
                    metric=VectorSearchAlgorithmMetric.COSINE,
                ),
            ),
            ExhaustiveKnnAlgorithmConfiguration(
                name="myExhaustiveKnn",
                kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
                parameters=ExhaustiveKnnParameters(
                    metric=VectorSearchAlgorithmMetric.COSINE
                ),
            ),
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
                vectorizer_name="myvectorizer"
            ),
            VectorSearchProfile(
                name="myExhaustiveKnnProfile",
                algorithm_configuration_name="myExhaustiveKnn",
            ),
        ],
        vectorizers=[  
            AzureOpenAIVectorizer(  
                vectorizer_name="myvectorizer",  
                kind="azureOpenAI",  
                parameters=AzureOpenAIVectorizerParameters(  
                    resource_url=os.environ["AZURE_OPENAI_ENDPOINT"],  
                    deployment_name=os.environ["AZURE_EMBEDDING_NAME"],
                    model_name=os.environ["AZURE_EMBEDDING_NAME"],
                    # Todo: there is some issue with managed identity for AI search
                    # authIdentity="/subscriptions/db4948d5-b90c-47d4-91b5-d8c4c43493d2/resourcegroups/rg-chainlit-agent/providers/Microsoft.ManagedIdentity/userAssignedIdentities/id-7r5g6is3dx73u"
                ),
            ),  
        ],  
    )

    # Create the semantic settings with the configuration
    semantic_search = SemanticSearch(configurations=[semantic_config])

    # Create the search index.
    index = SearchIndex(
        name=name,
        fields=fields,
        semantic_search=semantic_search,
        vector_search=vector_search,
    )

    return index

In [20]:
def chunk_text(text: str, max_tokens: int = 6500) -> List[str]:
    """
    Split text into chunks that fit within the token limit.
    Using a more conservative estimate of ~3 characters per token.
    """
    # More conservative estimate: 1 token ≈ 3 characters (down from 4)
    # And using 6000 tokens max instead of 7000 for extra safety
    max_chars = max_tokens * 3
    
    # Split by paragraphs first (double newlines)
    paragraphs = text.split('\n\n')
    
    chunks = []
    current_chunk = ""
    
    for paragraph in paragraphs:
        # If adding this paragraph would exceed the limit, start a new chunk
        if len(current_chunk) + len(paragraph) + 2 > max_chars and current_chunk:
            chunks.append(current_chunk.strip())
            current_chunk = paragraph
        else:
            if current_chunk:
                current_chunk += "\n\n" + paragraph
            else:
                current_chunk = paragraph
    
    # Add the last chunk if it exists
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    # Handle edge case where a single paragraph is too long
    final_chunks = []
    for chunk in chunks:
        if len(chunk) <= max_chars:
            final_chunks.append(chunk)
        else:
            # Split by sentences if paragraph is too long
            sentences = chunk.split('. ')
            temp_chunk = ""
            for sentence in sentences:
                if len(temp_chunk) + len(sentence) + 2 > max_chars and temp_chunk:
                    final_chunks.append(temp_chunk.strip())
                    temp_chunk = sentence
                else:
                    if temp_chunk:
                        temp_chunk += ". " + sentence
                    else:
                        temp_chunk = sentence
            if temp_chunk:
                final_chunks.append(temp_chunk.strip())
    
    # Final safety check - if any chunk is still too large, split it by words
    safe_chunks = []
    for chunk in final_chunks:
        if len(chunk) <= max_chars:
            safe_chunks.append(chunk)
        else:
            # Emergency word-level splitting
            words = chunk.split()
            temp_chunk = ""
            for word in words:
                if len(temp_chunk) + len(word) + 1 > max_chars and temp_chunk:
                    safe_chunks.append(temp_chunk.strip())
                    temp_chunk = word
                else:
                    if temp_chunk:
                        temp_chunk += " " + word
                    else:
                        temp_chunk = word
            if temp_chunk:
                safe_chunks.append(temp_chunk.strip())
    
    return safe_chunks

def gen_multi_format_documents(
    folder_path: str,
) -> List[Dict[str, any]]:
    """
    Process documents of multiple formats (MD, CSV, PDF, Excel, etc.)
    """
    openai_service_endoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    openai_deployment = os.environ["AZURE_EMBEDDING_NAME"]

    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")    
    client = AzureOpenAI(
        api_version="2023-07-01-preview",
        azure_endpoint=openai_service_endoint,
        azure_deployment=openai_deployment,
        azure_ad_token_provider=token_provider
    )

    items = []
    folder = Path(folder_path)
    
    # Supported file extensions
    supported_extensions = {'.md', '.txt', '.csv', '.pdf', '.xlsx', '.xls', '.json'}
    
    # Find all supported files in the folder and subfolders
    all_files = []
    for ext in supported_extensions:
        all_files.extend(folder.glob(f"**/*{ext}"))
    
    print(f"Found {len(all_files)} supported files to process")
    
    doc_id = 1
    for file_path in all_files:
        try:
            print(f"\nProcessing: {file_path.name} ({file_path.suffix})")
            
            # Get the appropriate processor for this file type
            processor = get_file_processor(file_path)
            content = processor(file_path)
            
            if not content or len(content.strip()) < 50:
                print(f"⚠️  Skipping {file_path.name} - insufficient content")
                continue
            
            # Determine document type and title
            file_extension = file_path.suffix.lower()
            document_type = {
                '.md': 'Markdown Document',
                '.txt': 'Text Document', 
                '.csv': 'CSV Data Table',
                '.pdf': 'PDF Document',
                '.xlsx': 'Excel Spreadsheet',
                '.xls': 'Excel Spreadsheet',
                '.json': 'JSON Data'
            }.get(file_extension, 'Unknown Document')
            
            # Extract title based on file type
            if file_extension == '.md':
                # Extract title from first heading or use filename
                title = file_path.stem
                lines = content.split('\n')
                for line in lines:
                    if line.startswith('# '):
                        title = line[2:].strip()
                        break
            elif file_extension == '.csv':
                # For CSV, try to extract a meaningful title from content
                title = file_path.stem
                # Look for title in first few lines
                lines = content.split('\n')[:5]
                for line in lines:
                    if len(line.strip()) > 10 and not line.startswith('Unnamed'):
                        title = line.strip()[:100]  # Limit title length
                        break
            else:
                title = file_path.stem
            
            # Preserve document metadata for referencing
            original_filename = file_path.name
            document_path = str(file_path.relative_to(folder))
            document_stem = file_path.stem
            
            # Split content into chunks
            content_chunks = chunk_text(content)
            
            print(f"Document: {title} - Split into {len(content_chunks)} chunks")
            
            # Process each chunk as a separate document
            for chunk_idx, chunk in enumerate(content_chunks):
                # Create unique ID for each chunk
                chunk_id = f"{doc_id}_{chunk_idx + 1}" if len(content_chunks) > 1 else str(doc_id)
                
                # Preserve original title for reference, add chunk info only for display
                display_title = title
                if len(content_chunks) > 1:
                    display_title = f"{title} (Part {chunk_idx + 1})"
                
                # Enhanced content with document reference information
                enhanced_content = f"Source Document: {original_filename}\n"
                enhanced_content += f"Document Type: {document_type}\n"
                if len(content_chunks) > 1:
                    enhanced_content += f"Document Section: Part {chunk_idx + 1} of {len(content_chunks)}\n"
                enhanced_content += f"Original Title: {title}\n\n{chunk}"
                
                url = f"/docs/{document_path.replace(file_extension, '').replace('/', '-').lower()}"
                if len(content_chunks) > 1:
                    url += f"-part-{chunk_idx + 1}"
                
                # Generate embeddings for enhanced content and title
                try:
                    print(f"Processing chunk {chunk_idx + 1}: {len(enhanced_content)} chars (estimated {len(enhanced_content)//3} tokens)")
                    
                    content_emb = client.embeddings.create(input=enhanced_content, model=openai_deployment)
                    title_emb = client.embeddings.create(input=display_title, model=openai_deployment)
                    
                    rec = {
                        "id": chunk_id,
                        "content": enhanced_content,  # Contains source reference
                        "filepath": document_path,     # Original relative path
                        "title": display_title,       # Display title with chunk info
                        "url": url,
                        "contentVector": content_emb.data[0].embedding,
                        "titleVector": title_emb.data[0].embedding,
                        # Additional metadata for better referencing
                        "originalFilename": original_filename,
                        "originalTitle": title,
                        "documentStem": document_stem,
                        "documentType": document_type,
                        "fileExtension": file_extension,
                        "chunkIndex": chunk_idx + 1,
                        "totalChunks": len(content_chunks),
                        "isChunked": len(content_chunks) > 1
                    }
                    items.append(rec)
                    print(f"✓ Successfully processed: {display_title}")
                    
                except Exception as e:
                    print(f"✗ Error processing chunk {chunk_idx + 1} of {title}: {e}")
                    print(f"  Enhanced content length: {len(enhanced_content)} characters (estimated {len(enhanced_content)//3} tokens)")
                    print(f"  Original chunk length: {len(chunk)} characters")
                    continue
            
            doc_id += 1
            
        except Exception as e:
            print(f"✗ Error processing file {file_path.name}: {e}")
            continue

    return items

# Keep the old function for backward compatibility
def gen_markdown_documents(folder_path: str) -> List[Dict[str, any]]:
    """Legacy function - use gen_multi_format_documents instead"""
    return gen_multi_format_documents(folder_path)

In [21]:
search_endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
index_name = os.environ["AZURE_SEARCH_INDEX"]

print(f"🔄 Starting fresh index creation pipeline...")
print(f"Index: {index_name}")
print(f"Endpoint: {search_endpoint}")

search_index_client = SearchIndexClient(
    search_endpoint, DefaultAzureCredential()
)

# Always delete existing index for fresh start (testing mode)
try:
    existing_index = search_index_client.get_index(index_name)
    print(f"🗑️ Deleting existing index '{index_name}'...")
    delete_index(search_index_client, index_name)
    print(f"✅ Index deleted successfully")
except Exception as e:
    print(f"ℹ️ Index '{index_name}' did not exist (this is fine)")

# Create fresh index with latest schema
print(f"🏗️ Creating new index '{index_name}' with latest schema...")
index = create_index_definition(index_name)
search_index_client.create_or_update_index(index)
print(f"✅ Fresh index '{index_name}' created successfully!")
print(f"📝 Schema includes all metadata fields: documentType, fileExtension, etc.")

🔄 Starting fresh index creation pipeline...
Index: parssedindexer1
Endpoint: https://srch-cqbrigubozxty.search.windows.net/
🗑️ Deleting existing index 'parssedindexer1'...
deleting index parssedindexer1
🗑️ Deleting existing index 'parssedindexer1'...
deleting index parssedindexer1
✅ Index deleted successfully
🏗️ Creating new index 'parssedindexer1' with latest schema...
✅ Index deleted successfully
🏗️ Creating new index 'parssedindexer1' with latest schema...
✅ Fresh index 'parssedindexer1' created successfully!
📝 Schema includes all metadata fields: documentType, fileExtension, etc.
✅ Fresh index 'parssedindexer1' created successfully!
📝 Schema includes all metadata fields: documentType, fileExtension, etc.


In [22]:
print(f"Processing documents for indexing...")

# Configuration for document processing
OVERWRITE_EXISTING = False  # Set to True to overwrite existing documents
CHECK_FOR_DUPLICATES = True  # Set to True to avoid processing already indexed files

# Change this path to point to your folder containing documents
# Now supports: .md, .txt, .csv, .pdf, .xlsx, .xls, .json files
document_folder = "./data/"  # Update this path as needed

# Create search client for document operations
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=DefaultAzureCredential(),
)

# Process all documents using the multi-format function
print(f"🔍 Supported file types: .md, .txt, .csv, .pdf, .xlsx, .xls, .json")
print(f"📂 Scanning folder: {document_folder}")

docs = gen_multi_format_documents(document_folder)

if len(docs) == 0:
    print("⚠️ No documents found to process")
    print(f"   Make sure your documents are in: {document_folder}")
    print(f"   Supported extensions: .md, .txt, .csv, .pdf, .xlsx, .xls, .json")
else:
    print(f"\n📤 Uploading {len(docs)} document chunks to fresh index...")
    
    # Upload in batches to handle large document sets
    batch_size = 50
    successful_batches = 0
    failed_batches = 0
    
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        try:
            result = search_client.upload_documents(batch)
            successful_batches += 1
            print(f"✅ Uploaded batch {i//batch_size + 1}: {len(batch)} documents")
        except Exception as e:
            failed_batches += 1
            print(f"❌ Error uploading batch {i//batch_size + 1}: {e}")
    
    print(f"\n🎉 Document indexing completed!")
    print(f"   📊 Total chunks processed: {len(docs)}")
    print(f"   ✅ Successful batches: {successful_batches}")
    if failed_batches > 0:
        print(f"   ❌ Failed batches: {failed_batches}")
    
    # Show summary by document type
    type_summary = {}
    for doc in docs:
        doc_type = doc.get("documentType", "Unknown")
        type_summary[doc_type] = type_summary.get(doc_type, 0) + 1
    
    print(f"\n📋 Documents processed by type:")
    for doc_type, count in sorted(type_summary.items()):
        print(f"   📄 {doc_type}: {count} chunks")

Processing documents for indexing...
🔍 Supported file types: .md, .txt, .csv, .pdf, .xlsx, .xls, .json
📂 Scanning folder: ./data/
Found 5 supported files to process

Processing: merged_statistic_id558217.csv (.csv)
Document: Statistik als Exceldatei - Split into 1 chunks
Processing chunk 1: 2156 chars (estimated 718 tokens)
✓ Successfully processed: Statistik als Exceldatei

Processing: Statista_2025_Ausländerkriminalität in Deutschland.pdf (.pdf)
✓ Successfully processed: Statistik als Exceldatei

Processing: Statista_2025_Ausländerkriminalität in Deutschland.pdf (.pdf)
Document: Statista_2025_Ausländerkriminalität in Deutschland - Split into 3 chunks
Processing chunk 1: 18773 chars (estimated 6257 tokens)
Document: Statista_2025_Ausländerkriminalität in Deutschland - Split into 3 chunks
Processing chunk 1: 18773 chars (estimated 6257 tokens)
✓ Successfully processed: Statista_2025_Ausländerkriminalität in Deutschland (Part 1)
Processing chunk 2: 19258 chars (estimated 6419 

In [23]:
# Utility functions for index management

def clean_german_text_simple(text: str) -> str:
    """Clean up common German encoding issues"""
    # Fix common UTF-8 encoding artifacts for German umlauts
    replacements = {
        'Ã¤': 'ä',
        'Ã¶': 'ö', 
        'Ã¼': 'ü',
        'Ã': 'ß',
        'Ã„': 'Ä',
        'Ã–': 'Ö',
        'Ãœ': 'Ü',
        'â€œ': '"',
        'â€': '"',
        'â€™': "'",
        'â€"': '–',
        'â€"': '—'
    }
    
    for bad, good in replacements.items():
        text = text.replace(bad, good)
    
    return text

def check_encoding_issues():
    """Check if there are encoding issues in the current index"""
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        # Get a few documents to check for encoding issues
        results = search_client.search("*", select=["id", "title", "content", "originalFilename"], top=10)
        
        encoding_issues = []
        for result in results:
            title = result.get("title", "")
            content = result.get("content", "")
            filename = result.get("originalFilename", "")
            
            # Check for common encoding artifacts
            if any(artifact in title + content for artifact in ['Ã¤', 'Ã¶', 'Ã¼', 'Ã', 'â€']):
                encoding_issues.append({
                    "id": result["id"],
                    "filename": filename,
                    "title": title,
                    "issues": [artifact for artifact in ['Ã¤', 'Ã¶', 'Ã¼', 'Ã'] if artifact in title + content]
                })
        
        if encoding_issues:
            print(f"🚨 Found {len(encoding_issues)} documents with encoding issues:")
            for issue in encoding_issues[:5]:  # Show first 5
                print(f"   📄 {issue['filename']}: {', '.join(issue['issues'])}")
            if len(encoding_issues) > 5:
                print(f"   ... and {len(encoding_issues) - 5} more")
            print(f"\n💡 Recommendation: Run the fresh index pipeline to fix encoding issues")
        else:
            print(f"✅ No encoding issues detected in current index")
            
        return len(encoding_issues)
        
    except Exception as e:
        print(f"❌ Error checking encoding: {e}")
        return -1

def get_index_stats():
    """Get statistics about the current fresh index"""
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        # Get total document count
        results = search_client.search("*", select=["id"], include_total_count=True)
        total_docs = results.get_count()
        
        # Get document type breakdown (schema is fresh, so these fields exist)
        file_results = search_client.search("*", select=["originalFilename", "documentType"])
        
        unique_files = set()
        doc_types = {}
        
        for result in file_results:
            if result.get("originalFilename"):
                unique_files.add(result["originalFilename"])
            if result.get("documentType"):
                doc_type = result["documentType"]
                doc_types[doc_type] = doc_types.get(doc_type, 0) + 1
        
        print(f"📊 Fresh Index Statistics:")
        print(f"   📄 Total documents/chunks: {total_docs}")
        print(f"   📁 Unique source files: {len(unique_files)}")
        print(f"   📈 Average chunks per file: {total_docs / len(unique_files) if unique_files else 0:.1f}")
        
        if doc_types:
            print(f"\n📋 Document Types:")
            for doc_type, count in sorted(doc_types.items()):
                print(f"   📄 {doc_type}: {count} chunks")
        
        # Check for encoding issues
        print(f"\n🔍 Checking for encoding issues...")
        check_encoding_issues()
        
        return total_docs, len(unique_files)
    except Exception as e:
        print(f"❌ Error getting index stats: {e}")
        return None, None

def delete_documents_by_filename(filename: str):
    """Delete all chunks from a specific source file"""
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        # Find all documents from this file
        results = search_client.search(f'originalFilename:"{filename}"', select=["id"])
        doc_ids = [result["id"] for result in results]
        
        if doc_ids:
            # Delete the documents
            delete_docs = [{"@search.action": "delete", "id": doc_id} for doc_id in doc_ids]
            search_client.upload_documents(delete_docs)
            print(f"🗑️ Deleted {len(doc_ids)} chunks from file '{filename}'")
        else:
            print(f"ℹ️ No documents found for file '{filename}'")
            
    except Exception as e:
        print(f"❌ Error deleting documents: {e}")

def delete_documents_by_type(document_type: str):
    """Delete all documents of a specific type (e.g., 'CSV Data Table')"""
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        results = search_client.search(f'documentType:"{document_type}"', select=["id"])
        doc_ids = [result["id"] for result in results]
        
        if doc_ids:
            # Delete the documents
            delete_docs = [{"@search.action": "delete", "id": doc_id} for doc_id in doc_ids]
            search_client.upload_documents(delete_docs)
            print(f"🗑️ Deleted {len(doc_ids)} chunks of type '{document_type}'")
        else:
            print(f"ℹ️ No documents found for type '{document_type}'")
            
    except Exception as e:
        print(f"❌ Error deleting documents: {e}")

def list_indexed_files():
    """List all files that have been indexed with their types"""
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        results = search_client.search("*", 
                                     select=["originalFilename", "documentType", "totalChunks", "fileExtension"])
        
        files_info = {}
        for result in results:
            filename = result.get("originalFilename", "Unknown")
            doc_type = result.get("documentType", "Unknown")
            chunks = result.get("totalChunks", 1)
            extension = result.get("fileExtension", "")
            
            if filename not in files_info:
                files_info[filename] = {
                    "type": doc_type,
                    "chunks": chunks,
                    "extension": extension
                }
        
        print("📁 Indexed Files:")
        for filename, info in sorted(files_info.items()):
            print(f"   📄 {filename} ({info['type']}, {info['chunks']} chunks)")
            
        return files_info
    except Exception as e:
        print(f"❌ Error listing files: {e}")
        return {}

def search_by_file_type(file_type: str, query: str = "*", top: int = 5):
    """Search within specific file types"""
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        # Use documentType filter (schema is fresh, so this field exists)
        filter_expr = f'documentType eq "{file_type}"'
        
        results = search_client.search(
            query, 
            filter=filter_expr,
            select=["title", "originalFilename", "documentType", "content"],
            top=top
        )
        
        print(f"🔍 Search results for '{query}' in {file_type} files:")
        for result in results:
            print(f"   📄 {result['title']} (from {result['originalFilename']})")
            print(f"      Preview: {result['content'][:200]}...")
            print()
            
    except Exception as e:
        print(f"❌ Error searching: {e}")

def test_vector_search(query: str = "product data", top: int = 3):
    """Test vector search functionality with the fresh index"""
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        print(f"🔍 Testing vector search for: '{query}'")
        
        # Perform hybrid search (text + vector)
        results = search_client.search(
            query,
            select=["title", "originalFilename", "documentType", "content"],
            top=top,
            query_type="semantic",
            semantic_configuration_name="default"
        )
        
        print(f"📋 Top {top} results:")
        for i, result in enumerate(results, 1):
            print(f"   {i}. {result['title']} ({result['documentType']})")
            print(f"      File: {result['originalFilename']}")
            print(f"      Preview: {result['content'][:150]}...")
            print()
            
    except Exception as e:
        print(f"❌ Error in vector search: {e}")

def test_german_search():
    """Test search with German terms to check for encoding issues"""
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        # Test with German terms that might have encoding issues
        test_queries = [
            "Kriminalität",
            "Ausländer", 
            "Straftäter",
            "Deutschland"
        ]
        
        print(f"🇩🇪 Testing German search terms:")
        for query in test_queries:
            print(f"\n🔍 Searching for: '{query}'")
            results = search_client.search(
                query,
                select=["title", "originalFilename", "content"],
                top=3
            )
            
            result_count = 0
            for result in results:
                result_count += 1
                title = result['title']
                filename = result['originalFilename']
                print(f"   📄 {title} (from {filename})")
            
            if result_count == 0:
                print(f"   ⚠️ No results found - possible encoding issues")
        
    except Exception as e:
        print(f"❌ Error in German search test: {e}")

# Show current statistics for the fresh index
print("🎯 Fresh Index Ready!")
get_index_stats()
print("\n📁 Files in index:")
list_indexed_files()
print("\n🇩🇪 Testing German search:")
test_german_search()

🎯 Fresh Index Ready!
📊 Fresh Index Statistics:
   📄 Total documents/chunks: 0
   📁 Unique source files: 0
   📈 Average chunks per file: 0.0

🔍 Checking for encoding issues...
📊 Fresh Index Statistics:
   📄 Total documents/chunks: 0
   📁 Unique source files: 0
   📈 Average chunks per file: 0.0

🔍 Checking for encoding issues...
🚨 Found 3 documents with encoding issues:
   📄 Baier_2015_Migration und KriminalitÃ¤t_converted.md: Ã¤, Ã
   📄 Baier_2015_Migration und KriminalitÃ¤t_converted.md: Ã¤, Ã
   📄 Baier_2015_Migration und KriminalitÃ¤t_converted.md: Ã¤, Ã

💡 Recommendation: Run the fresh index pipeline to fix encoding issues

📁 Files in index:
🚨 Found 3 documents with encoding issues:
   📄 Baier_2015_Migration und KriminalitÃ¤t_converted.md: Ã¤, Ã
   📄 Baier_2015_Migration und KriminalitÃ¤t_converted.md: Ã¤, Ã
   📄 Baier_2015_Migration und KriminalitÃ¤t_converted.md: Ã¤, Ã

💡 Recommendation: Run the fresh index pipeline to fix encoding issues

📁 Files in index:
📁 Indexed Files:
   📄 B

In [31]:
# Index Schema Update Helper

# Step 4: Test the Fresh Index with Sample Searches

def perform_test_searches():
    """Run sample searches to test the fresh index functionality"""
    
    print("🧪 Testing fresh index with sample searches...\n")
    
    # Test 1: General content search
    print("1️⃣ Testing general content search:")
    test_vector_search("data analysis", top=3)
    
    # Test 2: Search by document type if we have multiple types
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        # Get available document types
        type_results = search_client.search("*", select=["documentType"], top=100)
        doc_types = set()
        for result in type_results:
            if result.get("documentType"):
                doc_types.add(result["documentType"])
        
        if len(doc_types) > 1:
            print("2️⃣ Testing search by document type:")
            for doc_type in sorted(doc_types):
                print(f"\n   📄 Searching in {doc_type}:")
                search_by_file_type(doc_type, "*", top=2)
        else:
            print("2️⃣ Only one document type found, skipping type-specific search")
            
    except Exception as e:
        print(f"❌ Error in document type search test: {e}")
    
    # Test 3: Semantic search
    print("\n3️⃣ Testing semantic search:")
    try:
        results = search_client.search(
            "statistics and numbers",
            select=["title", "originalFilename", "content"],
            top=3,
            query_type="semantic",
            semantic_configuration_name="default"
        )
        
        print("   📊 Semantic search results:")
        for result in results:
            print(f"   • {result['title']} (from {result['originalFilename']})")
            
    except Exception as e:
        print(f"   ⚠️ Semantic search not fully configured: {e}")
    
    print("\n✅ Fresh index testing completed!")

# Run the test searches
perform_test_searches()

# Final summary
print("\n" + "="*60)
print("🎉 FRESH INDEX PIPELINE COMPLETED SUCCESSFULLY!")
print("="*60)
print("✅ Index deleted and recreated with latest schema")
print("✅ All documents processed and uploaded")  
print("✅ Utility functions ready for use")
print("✅ Sample searches tested")
print("\n💡 Next steps:")
print("   • Use the utility functions above to manage your index")
print("   • Run search_by_file_type() to search specific document types")
print("   • Use delete_documents_by_filename() to remove specific files")
print("   • The index is now ready for your application!")
print("="*60)

🧪 Testing fresh index with sample searches...

1️⃣ Testing general content search:
🔍 Testing vector search for: 'data analysis'
📋 Top 3 results:
2️⃣ Only one document type found, skipping type-specific search

3️⃣ Testing semantic search:
   📊 Semantic search results:

✅ Fresh index testing completed!

🎉 FRESH INDEX PIPELINE COMPLETED SUCCESSFULLY!
✅ Index deleted and recreated with latest schema
✅ All documents processed and uploaded
✅ Utility functions ready for use
✅ Sample searches tested

💡 Next steps:
   • Use the utility functions above to manage your index
   • Run search_by_file_type() to search specific document types
   • Use delete_documents_by_filename() to remove specific files
   • The index is now ready for your application!
2️⃣ Only one document type found, skipping type-specific search

3️⃣ Testing semantic search:
   📊 Semantic search results:

✅ Fresh index testing completed!

🎉 FRESH INDEX PIPELINE COMPLETED SUCCESSFULLY!
✅ Index deleted and recreated with latest s

In [25]:
# Quick Encoding and Content Check

def check_current_encoding_issues():
    """Check if there are encoding issues in the current index"""
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        # Search for documents that might contain the specific information
        results = search_client.search("ausländische Straftatverdächtige", select=["id", "title", "content", "originalFilename"], top=5)
        
        print("🔍 Searching for 'ausländische Straftatverdächtige':")
        found_results = False
        for result in results:
            found_results = True
            title = result.get("title", "")
            content = result.get("content", "")
            filename = result.get("originalFilename", "")
            
            print(f"\n📄 {filename}")
            print(f"   Title: {title}")
            
            # Check for encoding issues
            if any(artifact in title + content for artifact in ['Ã¤', 'Ã¶', 'Ã¼', 'Ã']):
                print(f"   ❌ Encoding issues found!")
                # Show the problematic content
                for artifact in ['Ã¤', 'Ã¶', 'Ã¼', 'Ã']:
                    if artifact in content:
                        print(f"      Found '{artifact}' in content")
            else:
                print(f"   ✅ No encoding issues")
            
            # Look for the specific statistic we want
            if "913.000" in content or "913,000" in content or "330.000" in content:
                print(f"   🎯 Contains target statistics!")
                # Show excerpt
                if "330.000" in content:
                    start = content.find("330.000") - 50
                    end = content.find("330.000") + 100
                    excerpt = content[max(0, start):end]
                    print(f"      Excerpt: ...{excerpt}...")
            
            print(f"   Content preview: {content[:200]}...")
        
        if not found_results:
            print("   ⚠️ No results found for German search term")
            
            # Try with encoding artifacts
            print(f"\n🔍 Trying with encoding artifacts:")
            artifact_results = search_client.search("auslÃ¤ndische", select=["id", "title", "originalFilename"], top=3)
            for result in artifact_results:
                print(f"   📄 {result.get('originalFilename', 'Unknown')}: {result.get('title', 'No title')}")
        
    except Exception as e:
        print(f"❌ Error checking encoding: {e}")

def test_csv_content_extraction():
    """Test how the current CSV file is being processed"""
    csv_file_path = Path("./data/merged_statistic_id558217.csv")
    
    if not csv_file_path.exists():
        print(f"❌ CSV file not found: {csv_file_path}")
        return
    
    print(f"📊 Testing CSV extraction from: {csv_file_path.name}")
    
    try:
        # Read the CSV file as it would be processed
        df = pd.read_csv(csv_file_path, encoding='utf-8')
        print(f"   Columns: {list(df.columns)}")
        print(f"   Shape: {df.shape}")
        
        # Look for the specific row with the statistics
        for idx, row in df.iterrows():
            row_text = ' '.join([str(val) for val in row.values if pd.notna(val) and str(val).strip()])
            if "330.000" in row_text or "330,000" in row_text:
                print(f"   🎯 Found target row {idx}: {row_text[:200]}...")
                break
        else:
            print(f"   ⚠️ Target statistics not found in processed content")
            
        # Show first few meaningful rows
        print(f"\n   First few meaningful rows:")
        for idx, row in df.iterrows():
            row_text = ' '.join([str(val) for val in row.values if pd.notna(val) and str(val).strip()])
            if len(row_text.strip()) > 20:
                print(f"   Row {idx}: {row_text[:150]}...")
                if idx >= 3:
                    break
        
    except Exception as e:
        print(f"❌ Error reading CSV: {e}")

# Run the checks
check_current_encoding_issues()
print("\n" + "="*50)
test_csv_content_extraction()

🔍 Searching for 'ausländische Straftatverdächtige':



📄 Statista_2025_Ausländerkriminalität in Deutschland.pdf
   Title: Statista_2025_Ausländerkriminalität in Deutschland (Part 1)
   ✅ No encoding issues
   🎯 Contains target statistics!
      Excerpt: ...engesetze in Deutschland wurden im Jahr 2024 rund 330.000 ausländische Straftatverdächtige von der Polizei ermittelt; vor allem mit Bezug auf Auslände...
   Content preview: Source Document: Statista_2025_Ausländerkriminalität in Deutschland.pdf
Document Type: PDF Document
Document Section: Part 1 of 3
Original Title: Statista_2025_Ausländerkriminalität in Deutschland...

📄 Baier_2015_Migration und KriminalitÃ¤t_converted.md
   Title: Baier_2015_Migration und KriminalitÃ¤t_converted (Part 1)
   ❌ Encoding issues found!
      Found 'Ã¤' in content
      Found 'Ã' in content
   Content preview: Source Document: Baier_2015_Migration und KriminalitÃ¤t_converted.md
Document Type: Markdown Document
Document Section: Part 1 of 3
Original Title: Baier_2015_Migration und KriminalitÃ¤t_co

In [26]:
# Focused Test for the Specific Issue

def test_specific_search():
    """Test search for the exact statistics mentioned in the logs"""
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        # Test searches that should find the data
        test_queries = [
            "ausländische Straftatverdächtige 2024",
            "330.000 ausländische", 
            "913.000 ausländische",
            "Straftatverdächtige Deutschland 2024",
            "Delikten gegen strafrechtliche Nebengesetze"
        ]
        
        for query in test_queries:
            print(f"🔍 Testing: '{query}'")
            results = search_client.search(query, select=["originalFilename", "title", "content"], top=3)
            
            found_any = False
            for result in results:
                found_any = True
                filename = result.get("originalFilename", "Unknown")
                title = result.get("title", "No title")
                content = result.get("content", "")
                
                print(f"   📄 {filename}")
                
                # Check for encoding issues in results
                if "Ã" in title or "Ã" in content:
                    print(f"      ❌ Encoding issues present")
                
                # Check if this contains our target stats
                if any(stat in content for stat in ["330.000", "330,000", "913.000", "913,000"]):
                    print(f"      🎯 Contains target statistics!")
                    
            if not found_any:
                print(f"      ⚠️ No results found")
            print()
        
        # Test if the CSV file is indexed at all
        print(f"📊 Checking if CSV file is indexed:")
        csv_results = search_client.search("merged_statistic_id558217", select=["originalFilename", "title"], top=5)
        
        csv_found = False
        for result in csv_results:
            csv_found = True
            print(f"   📄 Found: {result.get('originalFilename', 'Unknown')}")
        
        if not csv_found:
            print(f"   ⚠️ CSV file not found in index - may not be processed correctly")
            
    except Exception as e:
        print(f"❌ Error in search test: {e}")

def check_raw_csv_content():
    """Check the raw content of the CSV file to see what should be indexed"""
    csv_file = "./data/merged_statistic_id558217.csv"
    
    try:
        print(f"📝 Raw CSV content check:")
        with open(csv_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        print(f"   Total lines: {len(lines)}")
        
        # Find the line with our target data
        for i, line in enumerate(lines):
            if "330.000" in line or "Bei Delikten gegen strafrechtliche" in line:
                print(f"   🎯 Found target data at line {i+1}:")
                print(f"      {line.strip()}")
                
                # Check for encoding issues in the raw file
                if "Ã" in line:
                    print(f"      ❌ Raw file has encoding issues!")
                else:
                    print(f"      ✅ Raw file encoding looks correct")
                break
        else:
            print(f"   ⚠️ Target data not found in raw file")
        
    except Exception as e:
        print(f"❌ Error reading raw CSV: {e}")

# Run the focused tests
test_specific_search()
print("\n" + "="*40)
check_raw_csv_content()

🔍 Testing: 'ausländische Straftatverdächtige 2024'
   📄 Statista_2025_Ausländerkriminalität in Deutschland.pdf
      🎯 Contains target statistics!
   📄 Statista_2025_Ausländerkriminalität in Deutschland.pdf
   📄 merged_statistic_id558217.csv
      🎯 Contains target statistics!

🔍 Testing: '330.000 ausländische'
   📄 Baier_2015_Migration und KriminalitÃ¤t_converted.md
      ❌ Encoding issues present
   📄 Statista_2025_Ausländerkriminalität in Deutschland.pdf
      🎯 Contains target statistics!
   📄 merged_statistic_id558217.csv
      🎯 Contains target statistics!

🔍 Testing: '913.000 ausländische'
   📄 Baier_2015_Migration und KriminalitÃ¤t_converted.md
      ❌ Encoding issues present
   📄 Statista_2025_Ausländerkriminalität in Deutschland.pdf
      🎯 Contains target statistics!
   📄 merged_statistic_id558217.csv
      🎯 Contains target statistics!

🔍 Testing: 'Straftatverdächtige Deutschland 2024'
   📄 Statista_2025_Ausländerkriminalität in Deutschland.pdf
      🎯 Contains ta

In [27]:
# Improved CSV Processing for German Statistical Data

def process_german_statistics_csv(file_path: Path) -> str:
    """
    Specialized processor for German statistics CSV files like BKA crime statistics
    This handles messy CSV formats with metadata rows and German text
    """
    try:
        print(f"📊 Processing German statistics CSV: {file_path.name}")
        
        # Read the raw file with proper encoding
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        content_parts = []
        content_parts.append(f"German Statistical Data from {file_path.name}")
        content_parts.append("")
        
        # Extract meaningful rows with German text and numbers
        meaningful_rows = []
        data_rows = []
        
        for i, line in enumerate(lines):
            line = line.strip()
            
            # Skip empty lines and pure comma lines
            if not line or line.replace(',', '').strip() == '':
                continue
                
            # Parse CSV row
            try:
                row_parts = [part.strip().strip('"') for part in line.split(',')]
                row_text = ' '.join([part for part in row_parts if part and part != 'nan'])
                
                # Check if this row contains meaningful German text (more than just column headers)
                if len(row_text) > 20:
                    # Key statistical content - look for German keywords and numbers
                    if any(keyword in row_text.lower() for keyword in [
                        'straftat', 'verdächtig', 'kriminal', 'delikt', 'deutschland', 
                        'polizei', 'ermittelt', 'ausländer', 'rohheit', 'diebstahl'
                    ]):
                        meaningful_rows.append(f"Zeile {i+1}: {row_text}")
                        
                        # If it contains large numbers, it's likely key statistical data
                        if any(num in row_text for num in ['000', '.000', ',000']):
                            content_parts.append(f"📊 WICHTIGE STATISTIK: {row_text}")
                    
                    # Data rows with specific crime categories and numbers
                    elif any(crime_type in row_text for crime_type in [
                        'Strafrechtliche Nebengesetze', 'Rohheitsdelikte', 'Diebstahlsdelikte',
                        'Vermögens- und Fälschungsdelikte', 'Gewaltkriminalität', 'Rauschgiftkriminalität'
                    ]):
                        data_rows.append(row_text)
                        
            except Exception as e:
                # If CSV parsing fails, just treat as text
                if len(line) > 20 and any(keyword in line.lower() for keyword in [
                    'straftat', 'verdächtig', 'deutschland', 'polizei', 'ausländer'
                ]):
                    meaningful_rows.append(f"Text Zeile {i+1}: {line}")
        
        # Add meaningful content to the result
        if meaningful_rows:
            content_parts.append("BESCHREIBUNG UND KONTEXT:")
            content_parts.extend(meaningful_rows[:10])  # Limit to avoid too much metadata
            content_parts.append("")
        
        if data_rows:
            content_parts.append("STATISTISCHE DATEN:")
            content_parts.extend(data_rows)
            content_parts.append("")
        
        # Add source information
        content_parts.append("QUELLE: Bundeskriminalamt (BKA)")
        content_parts.append("JAHR: 2024")
        content_parts.append("REGION: Deutschland")
        
        result = '\n'.join(content_parts)
        
        # Clean up any encoding issues
        result = result.replace('Ã¤', 'ä').replace('Ã¶', 'ö').replace('Ã¼', 'ü').replace('Ã', 'ß')
        
        print(f"   ✅ Extracted {len(meaningful_rows)} description rows and {len(data_rows)} data rows")
        
        return result
        
    except Exception as e:
        print(f"   ❌ Error processing German statistics CSV: {e}")
        return f"Error processing German statistics file {file_path.name}: {str(e)}"

def test_improved_csv_processing():
    """Test the improved CSV processing on our specific file"""
    csv_file = Path("./data/merged_statistic_id558217.csv")
    
    if csv_file.exists():
        print("🧪 Testing improved CSV processing:")
        result = process_german_statistics_csv(csv_file)
        
        print("📋 Extracted content:")
        print("=" * 50)
        print(result)
        print("=" * 50)
        
        # Check if key information is present
        if "330.000" in result and "ausländische" in result:
            print("✅ Key statistics found in processed content!")
        else:
            print("⚠️ Key statistics may be missing from processed content")
            
        return result
    else:
        print("❌ CSV file not found for testing")
        return None

# Test the improved processing
test_improved_csv_processing()

🧪 Testing improved CSV processing:
📊 Processing German statistics CSV: merged_statistic_id558217.csv
   ✅ Extracted 19 description rows and 0 data rows
📋 Extracted content:
German Statistical Data from merged_statistic_id558217.csv

📊 WICHTIGE STATISTIK: Bei Delikten gegen strafrechtliche Nebengesetze in Deutschland wurden im Jahr 2024 rund 330.000 ausländische Straftatverdächtige von der Polizei ermittelt; vor allem mit Bezug auf Ausländergesetze. Damit waren dies die häufigsten Straftaten von Ausländern; gefolgt von Rohheits- und Freiheitsdelikten mit circa 246.000. Insgesamt gab es etwa 913.000 ausländische Tatverdächtige.**
BESCHREIBUNG UND KONTEXT:
Zeile 4: Anzahl der ausländischen Straftatverdächtigen in Deutschland nach Straftatengruppen im Jahr 2024
Zeile 9: Bei Delikten gegen strafrechtliche Nebengesetze in Deutschland wurden im Jahr 2024 rund 330.000 ausländische Straftatverdächtige von der Polizei ermittelt; vor allem mit Bezug auf Ausländergesetze. Damit waren dies die häuf

'German Statistical Data from merged_statistic_id558217.csv\n\n📊 WICHTIGE STATISTIK: Bei Delikten gegen strafrechtliche Nebengesetze in Deutschland wurden im Jahr 2024 rund 330.000 ausländische Straftatverdächtige von der Polizei ermittelt; vor allem mit Bezug auf Ausländergesetze. Damit waren dies die häufigsten Straftaten von Ausländern; gefolgt von Rohheits- und Freiheitsdelikten mit circa\xa0246.000. Insgesamt gab es etwa 913.000 ausländische Tatverdächtige.**\nBESCHREIBUNG UND KONTEXT:\nZeile 4: Anzahl der ausländischen Straftatverdächtigen in Deutschland nach Straftatengruppen im Jahr 2024\nZeile 9: Bei Delikten gegen strafrechtliche Nebengesetze in Deutschland wurden im Jahr 2024 rund 330.000 ausländische Straftatverdächtige von der Polizei ermittelt; vor allem mit Bezug auf Ausländergesetze. Damit waren dies die häufigsten Straftaten von Ausländern; gefolgt von Rohheits- und Freiheitsdelikten mit circa\xa0246.000. Insgesamt gab es etwa 913.000 ausländische Tatverdächtige.**\nZe

In [30]:
# Verification: Test that German search now works properly

print("🔍 VERIFICATION: Testing German search after encoding fix")
print("="*50)

try:
    search_client = SearchClient(
        endpoint=search_endpoint,
        index_name=index_name,
        credential=DefaultAzureCredential(),
    )
    
    # Test the exact question from your logs
    test_queries = [
        "im Jahr 2024 wie viel ausländische Straftatverdächtige von der Polizei ermittelt",
        "330.000 ausländische Straftatverdächtige", 
        "BKA Statistik Deutschland 2024",
        "ausländische Straftatverdächtige Deutschland"
    ]
    
    for query in test_queries:
        print(f"\n🔍 Testing: '{query}'")
        
        results = search_client.search(
            query,
            select=["title", "originalFilename", "content"],
            top=3,
            query_type="semantic",
            semantic_configuration_name="default"
        )
        
        found_results = False
        for result in results:
            found_results = True
            title = result.get("title", "No title")
            filename = result.get("originalFilename", "Unknown")
            content = result.get("content", "")
            
            print(f"   📄 {filename}")
            print(f"   📝 {title}")
            
            # Check if this contains the key statistics
            if "330.000" in content:
                print(f"   🎯 ✅ CONTAINS KEY STATISTIC: 330.000")
            if "913.000" in content:
                print(f"   🎯 ✅ CONTAINS TOTAL: 913.000")
            if "ausländische" in content and "Straftatverdächtige" in content:
                print(f"   🎯 ✅ CONTAINS GERMAN TERMS (no encoding artifacts)")
            
            # Show a preview
            preview = content[:300].replace('\n', ' ')
            print(f"   📄 Preview: {preview}...")
            
        if not found_results:
            print(f"   ❌ No results found")
    
    # Final test: Check if there are any encoding artifacts left
    print(f"\n🔍 Checking for remaining encoding artifacts:")
    artifact_search = search_client.search("Ã", select=["title", "originalFilename"], top=5)
    
    artifacts_found = False
    for result in artifact_search:
        artifacts_found = True
        print(f"   ⚠️ Still has artifacts: {result.get('originalFilename', 'Unknown')}")
    
    if not artifacts_found:
        print(f"   ✅ No encoding artifacts found in index")
    
    print(f"\n🎉 VERIFICATION COMPLETE!")
    print(f"   Your backend should now properly find German statistical data")
    print(f"   The assistant should be able to answer questions about 2024 crime statistics")
    
except Exception as e:
    print(f"❌ Verification error: {e}")

# Show final index status
print(f"\n📊 Final Index Status:")
try:
    results = search_client.search("*", select=["id"], include_total_count=True)
    total_docs = results.get_count()
    print(f"   📄 Total documents in index: {total_docs}")
    
    # Check if our specific document is there
    bka_results = search_client.search("BKA", select=["originalFilename", "title"])
    for result in bka_results:
        print(f"   📊 Found: {result.get('title', 'No title')}")
        
except Exception as e:
    print(f"   ❌ Status check error: {e}")

🔍 VERIFICATION: Testing German search after encoding fix

🔍 Testing: 'im Jahr 2024 wie viel ausländische Straftatverdächtige von der Polizei ermittelt'
   📄 merged_statistic_id558217.csv
   📝 BKA Statistik 2024: Ausländische Straftatverdächtige Deutschland
   🎯 ✅ CONTAINS KEY STATISTIC: 330.000
   🎯 ✅ CONTAINS TOTAL: 913.000
   🎯 ✅ CONTAINS GERMAN TERMS (no encoding artifacts)
   📄 Preview: BKA Kriminalstatistik 2024 - Ausländische Straftatverdächtige  HAUPTSTATISTIK:  Bei Delikten gegen strafrechtliche Nebengesetze in Deutschland wurden im Jahr 2024 rund 330.000 ausländische Straftatverdächtige von der Polizei ermittelt; vor allem mit Bezug auf Ausländergesetze. Damit waren dies die h...

🔍 Testing: '330.000 ausländische Straftatverdächtige'
   📄 merged_statistic_id558217.csv
   📝 BKA Statistik 2024: Ausländische Straftatverdächtige Deutschland
   🎯 ✅ CONTAINS KEY STATISTIC: 330.000
   🎯 ✅ CONTAINS TOTAL: 913.000
   🎯 ✅ CONTAINS GERMAN TERMS (no encoding artifacts)
   📄 Preview: BKA K

In [32]:
# DEBUG: Check what files are actually being found and processed

def debug_file_discovery():
    """Debug what files are being found in the data directory"""
    from pathlib import Path
    
    folder = Path("./data/")
    supported_extensions = {'.md', '.txt', '.csv', '.pdf', '.xlsx', '.xls', '.json'}
    
    print(f"🔍 Debugging file discovery in: {folder.absolute()}")
    print(f"📁 Folder exists: {folder.exists()}")
    
    if folder.exists():
        print(f"📂 Contents of {folder}:")
        for item in folder.iterdir():
            print(f"   {item.name} ({'DIR' if item.is_dir() else 'FILE'})")
    
    # Find all supported files
    all_files = []
    for ext in supported_extensions:
        found_files = list(folder.glob(f"**/*{ext}"))
        if found_files:
            print(f"\n📄 Files with extension {ext}:")
            for f in found_files:
                print(f"   {f}")
                all_files.extend(found_files)
    
    print(f"\n📊 Total supported files found: {len(all_files)}")
    
    # Test processing each file
    print(f"\n🧪 Testing file processing:")
    for file_path in all_files:
        try:
            processor = get_file_processor(file_path)
            print(f"   📄 {file_path.name}: processor = {processor.__name__}")
            
            # Try to process
            content = processor(file_path)
            if content:
                content_len = len(content.strip())
                print(f"      ✅ Content length: {content_len} characters")
                if content_len < 50:
                    print(f"      ⚠️ Content too short - would be skipped")
            else:
                print(f"      ❌ No content returned")
                
        except Exception as e:
            print(f"      ❌ Error processing {file_path.name}: {e}")

debug_file_discovery()

🔍 Debugging file discovery in: /workspaces/chainlit-agent/data/product_info/data
📁 Folder exists: True
📂 Contents of data:
   merged_statistic_id558217.csv (FILE)
   Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md (FILE)
   Baier_2015_Migration und KriminalitÃ¤t_converted.md (FILE)
   City of Munich_2017_Flugblatt_Kriminalitaet_converted.md (FILE)
   Statista_2025_Ausländerkriminalität in Deutschland.pdf (FILE)

📄 Files with extension .csv:
   data/merged_statistic_id558217.csv

📄 Files with extension .pdf:
   data/Statista_2025_Ausländerkriminalität in Deutschland.pdf

📄 Files with extension .md:
   data/Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md
   data/Baier_2015_Migration und KriminalitÃ¤t_converted.md
   data/City of Munich_2017_Flugblatt_Kriminalitaet_converted.md

📊 Total supported files found: 11

🧪 Testing file processing:
   📄 merged_statistic_id558217.csv: processor = process_csv_file
      ✅ Content length: 2037 characters
   📄 Statista_2025_Ausla

In [33]:
# Check what's actually in the index

def check_index_contents():
    """Check what documents are actually in the index"""
    try:
        search_client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=DefaultAzureCredential(),
        )
        
        # Get all documents
        results = search_client.search("*", 
                                     select=["id", "originalFilename", "documentType", "title", "totalChunks"], 
                                     top=50)
        
        files_summary = {}
        total_docs = 0
        
        for result in results:
            total_docs += 1
            filename = result.get("originalFilename", "Unknown")
            doc_type = result.get("documentType", "Unknown")
            title = result.get("title", "No title")
            total_chunks = result.get("totalChunks", 1)
            
            if filename not in files_summary:
                files_summary[filename] = {
                    "type": doc_type,
                    "chunks": total_chunks,
                    "sample_title": title
                }
        
        print(f"📊 Total documents in index: {total_docs}")
        print(f"📁 Files represented: {len(files_summary)}")
        print(f"\n📋 Files in index:")
        
        for filename, info in sorted(files_summary.items()):
            print(f"   📄 {filename}")
            print(f"      Type: {info['type']}")
            print(f"      Chunks: {info['chunks']}")
            print(f"      Sample title: {info['sample_title'][:60]}...")
            print()
            
        # Check which files from the data directory are missing
        print(f"🔍 Checking for missing files:")
        expected_files = [
            "merged_statistic_id558217.csv",
            "Baier_2015_Migration und KriminalitÃ¤t_converted.md",
            "City of Munich_2017_Flugblatt_Kriminalitaet_converted.md",
            "Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md",
            "Statista_2025_Ausländerkriminalität in Deutschland.pdf",
            "product_reviews.csv",
            "product_sales.csv", 
            "products.csv"
        ]
        
        indexed_files = set(files_summary.keys())
        missing_files = []
        
        for expected in expected_files:
            if expected not in indexed_files:
                missing_files.append(expected)
        
        if missing_files:
            print(f"❌ Missing files:")
            for missing in missing_files:
                print(f"   🚫 {missing}")
        else:
            print(f"✅ All expected files are indexed")
            
    except Exception as e:
        print(f"❌ Error checking index: {e}")

check_index_contents()

📊 Total documents in index: 1
📁 Files represented: 1

📋 Files in index:
   📄 merged_statistic_id558217.csv
      Type: German Crime Statistics
      Chunks: 1
      Sample title: BKA Statistik 2024: Ausländische Straftatverdächtige Deutsch...

🔍 Checking for missing files:
❌ Missing files:
   🚫 Baier_2015_Migration und KriminalitÃ¤t_converted.md
   🚫 City of Munich_2017_Flugblatt_Kriminalitaet_converted.md
   🚫 Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md
   🚫 Statista_2025_Ausländerkriminalität in Deutschland.pdf
   🚫 product_reviews.csv
   🚫 product_sales.csv
   🚫 products.csv


In [34]:
# COMPREHENSIVE DEBUG: Test what gen_multi_format_documents actually does

def debug_gen_multi_format_documents():
    """Debug version of gen_multi_format_documents to see what's happening"""
    import os
    from pathlib import Path
    
    folder_path = "./data/"
    
    print(f"🐛 DEBUG: Starting debug of gen_multi_format_documents")
    print(f"📁 Current working directory: {os.getcwd()}")
    print(f"🎯 Target folder: {folder_path}")
    
    folder = Path(folder_path)
    print(f"📂 Folder absolute path: {folder.absolute()}")
    print(f"📂 Folder exists: {folder.exists()}")
    
    if not folder.exists():
        print(f"❌ Folder does not exist!")
        return
    
    # Check what's in the folder
    print(f"\n📋 Contents of {folder}:")
    for item in folder.iterdir():
        if item.is_file():
            print(f"   📄 {item.name} ({item.suffix})")
        else:
            print(f"   📁 {item.name}/")
    
    # Find supported files
    supported_extensions = {'.md', '.txt', '.csv', '.pdf', '.xlsx', '.xls', '.json'}
    all_files = []
    
    print(f"\n🔍 Looking for files with supported extensions:")
    for ext in supported_extensions:
        found_files = list(folder.glob(f"**/*{ext}"))
        if found_files:
            print(f"   {ext}: {len(found_files)} files")
            for f in found_files:
                print(f"      📄 {f}")
                all_files.append(f)
        else:
            print(f"   {ext}: 0 files")
    
    print(f"\n📊 Total files to process: {len(all_files)}")
    
    # Try to process just the first file to see what happens
    if all_files:
        test_file = all_files[0]
        print(f"\n🧪 Testing processing of first file: {test_file.name}")
        
        try:
            processor = get_file_processor(test_file)
            print(f"   📋 Processor function: {processor.__name__}")
            
            content = processor(test_file)
            if content:
                print(f"   ✅ Content extracted: {len(content)} characters")
                if len(content.strip()) < 50:
                    print(f"   ⚠️ Content too short, would be skipped")
                else:
                    print(f"   ✅ Content sufficient for processing")
            else:
                print(f"   ❌ No content extracted")
                
        except Exception as e:
            print(f"   ❌ Error processing: {e}")
            import traceback
            traceback.print_exc()
    
    # Now let's see what happens if we call the actual function
    print(f"\n🚀 Calling actual gen_multi_format_documents...")
    try:
        result_docs = gen_multi_format_documents(folder_path)
        print(f"   ✅ Function returned {len(result_docs)} documents")
        
        if result_docs:
            print(f"   📋 Document details:")
            for i, doc in enumerate(result_docs[:3]):  # Show first 3
                print(f"      {i+1}. {doc.get('originalFilename', 'Unknown')} ({doc.get('documentType', 'Unknown')})")
                print(f"         Title: {doc.get('title', 'No title')[:50]}...")
                
    except Exception as e:
        print(f"   ❌ Error in gen_multi_format_documents: {e}")
        import traceback
        traceback.print_exc()

debug_gen_multi_format_documents()

🐛 DEBUG: Starting debug of gen_multi_format_documents
📁 Current working directory: /workspaces/chainlit-agent/data/product_info
🎯 Target folder: ./data/
📂 Folder absolute path: /workspaces/chainlit-agent/data/product_info/data
📂 Folder exists: True

📋 Contents of data:
   📄 merged_statistic_id558217.csv (.csv)
   📄 Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md (.md)
   📄 Baier_2015_Migration und KriminalitÃ¤t_converted.md (.md)
   📄 City of Munich_2017_Flugblatt_Kriminalitaet_converted.md (.md)
   📄 Statista_2025_Ausländerkriminalität in Deutschland.pdf (.pdf)

🔍 Looking for files with supported extensions:
   .txt: 0 files
   .json: 0 files
   .csv: 1 files
      📄 data/merged_statistic_id558217.csv
   .pdf: 1 files
      📄 data/Statista_2025_Ausländerkriminalität in Deutschland.pdf
   .xls: 0 files
   .xlsx: 0 files
   .md: 3 files
      📄 data/Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md
      📄 data/Baier_2015_Migration und KriminalitÃ¤t_converted.md
    

In [35]:
# Check what's currently in the 'docs' variable

print(f"🧐 Checking current 'docs' variable...")
print(f"   Type: {type(docs)}")
print(f"   Length: {len(docs)}")

if docs:
    print(f"\n📋 First few documents in 'docs':")
    for i, doc in enumerate(docs[:5]):
        print(f"   {i+1}. File: {doc.get('originalFilename', 'Unknown')}")
        print(f"      Type: {doc.get('documentType', 'Unknown')}")
        print(f"      Title: {doc.get('title', 'No title')[:50]}...")
        print(f"      Content preview: {doc.get('content', 'No content')[:80]}...")
        print()
    
    # Check file distribution
    file_counts = {}
    for doc in docs:
        filename = doc.get('originalFilename', 'Unknown')
        file_counts[filename] = file_counts.get(filename, 0) + 1
    
    print(f"📊 Document chunks by file:")
    for filename, count in sorted(file_counts.items()):
        print(f"   📄 {filename}: {count} chunks")
else:
    print("   ❌ 'docs' variable is empty!")

🧐 Checking current 'docs' variable...
   Type: <class 'list'>
   Length: 11

📋 First few documents in 'docs':
   1. File: merged_statistic_id558217.csv
      Type: CSV Data Table
      Title: Statistik als Exceldatei...
      Content preview: Source Document: merged_statistic_id558217.csv
Document Type: CSV Data Table
Ori...

   2. File: Statista_2025_Ausländerkriminalität in Deutschland.pdf
      Type: PDF Document
      Title: Statista_2025_Ausländerkriminalität in Deutschla...
      Content preview: Source Document: Statista_2025_Ausländerkriminalität in Deutschland.pdf
Docume...

   3. File: Statista_2025_Ausländerkriminalität in Deutschland.pdf
      Type: PDF Document
      Title: Statista_2025_Ausländerkriminalität in Deutschla...
      Content preview: Source Document: Statista_2025_Ausländerkriminalität in Deutschland.pdf
Docume...

   4. File: Statista_2025_Ausländerkriminalität in Deutschland.pdf
      Type: PDF Document
      Title: Statista_2025_Ausländerkri

In [36]:
# TEST UPLOAD: Try to upload the docs that are currently in memory

print(f"🧪 Testing upload of current documents...")
print(f"📊 Current docs count: {len(docs)}")

if len(docs) > 0:
    print(f"\n🔄 Attempting to upload {len(docs)} documents to index...")
    
    # Create search client  
    search_client = SearchClient(
        endpoint=search_endpoint,
        index_name=index_name,
        credential=DefaultAzureCredential(),
    )
    
    # Upload in small batches for testing
    batch_size = 5
    successful_batches = 0
    failed_batches = 0
    
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        try:
            print(f"   📤 Uploading batch {i//batch_size + 1}: {len(batch)} documents")
            result = search_client.upload_documents(batch)
            successful_batches += 1
            print(f"   ✅ Batch {i//batch_size + 1} uploaded successfully")
            
            # Show which files were in this batch
            batch_files = [doc.get('originalFilename', 'Unknown') for doc in batch]
            print(f"      Files: {', '.join(set(batch_files))}")
            
        except Exception as e:
            failed_batches += 1
            print(f"   ❌ Error uploading batch {i//batch_size + 1}: {e}")
            
            # Show details of the failed batch
            print(f"      Batch contents:")
            for j, doc in enumerate(batch):
                print(f"        {j+1}. {doc.get('originalFilename', 'Unknown')} - {doc.get('title', 'No title')[:40]}...")
    
    print(f"\n📊 Upload test results:")
    print(f"   ✅ Successful batches: {successful_batches}")
    print(f"   ❌ Failed batches: {failed_batches}")
    
    # Check index after upload
    results = search_client.search("*", select=["originalFilename"], top=20)
    indexed_files = set()
    total_in_index = 0
    for result in results:
        total_in_index += 1
        if result.get("originalFilename"):
            indexed_files.add(result["originalFilename"])
    
    print(f"\n📋 After upload test - Index contains:")
    print(f"   📊 Total documents: {total_in_index}")
    print(f"   📁 Unique files: {len(indexed_files)}")
    for filename in sorted(indexed_files):
        print(f"      📄 {filename}")
        
else:
    print("❌ No documents to upload!")

🧪 Testing upload of current documents...
📊 Current docs count: 11

🔄 Attempting to upload 11 documents to index...
   📤 Uploading batch 1: 5 documents
   ✅ Batch 1 uploaded successfully
      Files: Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md, merged_statistic_id558217.csv, Statista_2025_Ausländerkriminalität in Deutschland.pdf
   📤 Uploading batch 2: 5 documents
   ✅ Batch 2 uploaded successfully
      Files: Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md, Baier_2015_Migration und KriminalitÃ¤t_converted.md
   📤 Uploading batch 3: 1 documents
   ✅ Batch 3 uploaded successfully
      Files: City of Munich_2017_Flugblatt_Kriminalitaet_converted.md

📊 Upload test results:
   ✅ Successful batches: 3
   ❌ Failed batches: 0

📋 After upload test - Index contains:
   📊 Total documents: 6
   📁 Unique files: 3
      📄 Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md
      📄 Statista_2025_Ausländerkriminalität in Deutschland.pdf
      📄 merged_statistic_id55821

In [37]:
# DEBUG: Check document IDs for duplicates

print(f"🔍 Checking document IDs in index...")

search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=DefaultAzureCredential(),
)

# Get all documents with their IDs and filenames
results = search_client.search("*", select=["id", "originalFilename", "chunkIndex", "totalChunks"], top=50)

documents = []
id_counts = {}
file_chunks = {}

for result in results:
    doc_id = result.get("id", "No ID")
    filename = result.get("originalFilename", "Unknown")
    chunk_idx = result.get("chunkIndex", 0)
    total_chunks = result.get("totalChunks", 1)
    
    documents.append({
        "id": doc_id,
        "filename": filename,
        "chunk": chunk_idx,
        "total": total_chunks
    })
    
    # Count ID frequency
    id_counts[doc_id] = id_counts.get(doc_id, 0) + 1
    
    # Track file chunks
    if filename not in file_chunks:
        file_chunks[filename] = {"expected": total_chunks, "found": []}
    file_chunks[filename]["found"].append(chunk_idx)

print(f"📊 Total documents in index: {len(documents)}")
print(f"\n📋 Documents by file:")
for filename, info in sorted(file_chunks.items()):
    expected = info["expected"]
    found = sorted(info["found"])
    print(f"   📄 {filename}:")
    print(f"      Expected chunks: {expected}")
    print(f"      Found chunks: {len(found)} -> {found}")
    if len(found) != expected:
        missing = [i for i in range(1, expected + 1) if i not in found]
        if missing:
            print(f"      ❌ Missing chunks: {missing}")

# Check for duplicate IDs
duplicate_ids = {doc_id: count for doc_id, count in id_counts.items() if count > 1}
if duplicate_ids:
    print(f"\n❌ Found duplicate IDs:")
    for doc_id, count in duplicate_ids.items():
        print(f"   🔄 ID '{doc_id}' appears {count} times")
        
    # Show which documents have the same ID
    for doc_id in duplicate_ids:
        matching_docs = [doc for doc in documents if doc["id"] == doc_id]
        print(f"\n   📋 Documents with ID '{doc_id}':")
        for doc in matching_docs:
            print(f"      📄 {doc['filename']} (chunk {doc['chunk']}/{doc['total']})")
else:
    print(f"\n✅ No duplicate IDs found")

# Check the original docs variable for ID patterns
print(f"\n🧐 Checking original 'docs' variable for ID patterns:")
doc_ids_in_memory = [doc.get("id", "No ID") for doc in docs]
memory_id_counts = {}
for doc_id in doc_ids_in_memory:
    memory_id_counts[doc_id] = memory_id_counts.get(doc_id, 0) + 1

duplicate_memory_ids = {doc_id: count for doc_id, count in memory_id_counts.items() if count > 1}
if duplicate_memory_ids:
    print(f"❌ Found duplicate IDs in memory:")
    for doc_id, count in duplicate_memory_ids.items():
        print(f"   🔄 ID '{doc_id}' appears {count} times in docs variable")
        
    # Show which documents in memory have the same ID
    for doc_id in duplicate_memory_ids:
        matching_docs = [(i, doc) for i, doc in enumerate(docs) if doc.get("id") == doc_id]
        print(f"\n   📋 Documents in memory with ID '{doc_id}':")
        for i, doc in matching_docs:
            print(f"      {i}. {doc.get('originalFilename', 'Unknown')} (chunk {doc.get('chunkIndex', '?')}/{doc.get('totalChunks', '?')})")
else:
    print(f"✅ No duplicate IDs in memory docs")

🔍 Checking document IDs in index...
📊 Total documents in index: 12

📋 Documents by file:
   📄 Baier_2015_Migration und KriminalitÃ¤t_converted.md:
      Expected chunks: 3
      Found chunks: 3 -> [1, 2, 3]
   📄 City of Munich_2017_Flugblatt_Kriminalitaet_converted.md:
      Expected chunks: 1
      Found chunks: 1 -> [1]
   📄 Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md:
      Expected chunks: 3
      Found chunks: 3 -> [1, 2, 3]
   📄 Statista_2025_Ausländerkriminalität in Deutschland.pdf:
      Expected chunks: 3
      Found chunks: 3 -> [1, 2, 3]
   📄 merged_statistic_id558217.csv:
      Expected chunks: 1
      Found chunks: 2 -> [1, 1]

✅ No duplicate IDs found

🧐 Checking original 'docs' variable for ID patterns:
✅ No duplicate IDs in memory docs


In [38]:
# SOLUTION: Clean index and re-run the complete pipeline

print("🧹 Cleaning index and re-running complete pipeline...")

# Step 1: Delete existing index
search_index_client = SearchIndexClient(
    search_endpoint, DefaultAzureCredential()
)

try:
    print("🗑️ Deleting existing index...")
    search_index_client.delete_index(index_name)
    print("✅ Index deleted successfully")
except Exception as e:
    print(f"ℹ️ Index deletion result: {e}")

# Step 2: Create fresh index
print("🏗️ Creating fresh index...")
index = create_index_definition(index_name)
search_index_client.create_or_update_index(index)
print("✅ Fresh index created successfully")

# Step 3: Process documents
print("📂 Processing documents...")
docs = gen_multi_format_documents("./data/")
print(f"📊 Processed {len(docs)} document chunks")

# Step 4: Upload documents
if len(docs) > 0:
    print("📤 Uploading documents to index...")
    
    search_client = SearchClient(
        endpoint=search_endpoint,
        index_name=index_name,
        credential=DefaultAzureCredential(),
    )
    
    # Upload in batches
    batch_size = 50
    successful_batches = 0
    failed_batches = 0
    
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        try:
            result = search_client.upload_documents(batch)
            successful_batches += 1
            print(f"✅ Uploaded batch {i//batch_size + 1}: {len(batch)} documents")
        except Exception as e:
            failed_batches += 1
            print(f"❌ Error uploading batch {i//batch_size + 1}: {e}")
    
    print(f"\n📊 Upload completed:")
    print(f"   ✅ Successful batches: {successful_batches}")
    if failed_batches > 0:
        print(f"   ❌ Failed batches: {failed_batches}")
    
    # Step 5: Verify the result
    print("\n🔍 Verifying index contents...")
    results = search_client.search("*", select=["originalFilename", "documentType"], top=50)
    
    files_summary = {}
    total_docs = 0
    
    for result in results:
        total_docs += 1
        filename = result.get("originalFilename", "Unknown")
        doc_type = result.get("documentType", "Unknown")
        
        if filename not in files_summary:
            files_summary[filename] = {"type": doc_type, "count": 0}
        files_summary[filename]["count"] += 1
    
    print(f"📊 Final index contains {total_docs} documents from {len(files_summary)} files:")
    for filename, info in sorted(files_summary.items()):
        print(f"   📄 {filename}: {info['count']} chunks ({info['type']})")
    
    print(f"\n✅ Pipeline completed successfully!")
    print(f"🎉 All {len(files_summary)} files have been indexed with {total_docs} total chunks")
    
else:
    print("❌ No documents were processed!")

🧹 Cleaning index and re-running complete pipeline...
🗑️ Deleting existing index...
✅ Index deleted successfully
🏗️ Creating fresh index...
✅ Fresh index created successfully
📂 Processing documents...
Found 5 supported files to process

Processing: merged_statistic_id558217.csv (.csv)
Document: Statistik als Exceldatei - Split into 1 chunks
Processing chunk 1: 2156 chars (estimated 718 tokens)
✓ Successfully processed: Statistik als Exceldatei

Processing: Statista_2025_Ausländerkriminalität in Deutschland.pdf (.pdf)
Document: Statista_2025_Ausländerkriminalität in Deutschland - Split into 3 chunks
Processing chunk 1: 18773 chars (estimated 6257 tokens)
✓ Successfully processed: Statista_2025_Ausländerkriminalität in Deutschland (Part 1)
Processing chunk 2: 19258 chars (estimated 6419 tokens)
✓ Successfully processed: Statista_2025_Ausländerkriminalität in Deutschland (Part 2)
Processing chunk 3: 3244 chars (estimated 1081 tokens)
✓ Successfully processed: Statista_2025_Ausländ

In [39]:
# FINAL VERIFICATION: Confirm all files are properly indexed

print("🔍 Final verification of the indexed content...")

search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=DefaultAzureCredential(),
)

# Get comprehensive stats
results = search_client.search("*", select=["originalFilename", "documentType", "chunkIndex", "totalChunks"], top=50)

file_details = {}
total_documents = 0

for result in results:
    total_documents += 1
    filename = result.get("originalFilename", "Unknown")
    doc_type = result.get("documentType", "Unknown")
    chunk_idx = result.get("chunkIndex", 1)
    total_chunks = result.get("totalChunks", 1)
    
    if filename not in file_details:
        file_details[filename] = {
            "type": doc_type,
            "total_chunks": total_chunks,
            "found_chunks": [],
            "chunks_indexed": 0
        }
    
    file_details[filename]["found_chunks"].append(chunk_idx)
    file_details[filename]["chunks_indexed"] += 1

print(f"📊 INDEX SUMMARY:")
print(f"   Total documents indexed: {total_documents}")
print(f"   Unique files: {len(file_details)}")
print()

print(f"📋 DETAILED FILE BREAKDOWN:")
all_files_complete = True

for filename, details in sorted(file_details.items()):
    expected = details["total_chunks"]
    found = len(details["found_chunks"])
    chunks_found = sorted(details["found_chunks"])
    
    status = "✅" if found == expected else "⚠️"
    if found != expected:
        all_files_complete = False
    
    print(f"   {status} {filename}")
    print(f"      Type: {details['type']}")
    print(f"      Expected chunks: {expected}")
    print(f"      Indexed chunks: {found}")
    
    if found != expected:
        expected_chunks = list(range(1, expected + 1))
        missing = [i for i in expected_chunks if i not in chunks_found]
        duplicates = [i for i in chunks_found if chunks_found.count(i) > 1]
        
        if missing:
            print(f"      ❌ Missing chunks: {missing}")
        if duplicates:
            print(f"      🔄 Duplicate chunks: {duplicates}")
    print()

# Expected files check
expected_files = [
    "merged_statistic_id558217.csv",
    "Baier_2015_Migration und KriminalitÃ¤t_converted.md", 
    "City of Munich_2017_Flugblatt_Kriminalitaet_converted.md",
    "Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md",
    "Statista_2025_Ausländerkriminalität in Deutschland.pdf"
]

indexed_files = set(file_details.keys())
missing_files = [f for f in expected_files if f not in indexed_files]

if missing_files:
    print(f"❌ MISSING FILES:")
    for missing in missing_files:
        print(f"   🚫 {missing}")
    all_files_complete = False
else:
    print(f"✅ ALL EXPECTED FILES ARE INDEXED")

# Test German search
print(f"\n🧪 TESTING GERMAN SEARCH:")
german_results = search_client.search("ausländische Straftatverdächtige", top=3)
german_found = False
for result in german_results:
    german_found = True
    filename = result.get("originalFilename", "Unknown")
    title = result.get("title", "No title")
    print(f"   📄 {filename}: {title[:60]}...")

if not german_found:
    print(f"   ⚠️ No results found for German search term")

# Final status
print(f"\n{'='*60}")
if all_files_complete and not missing_files and german_found:
    print(f"🎉 SUCCESS: All files properly indexed and searchable!")
    print(f"✅ {len(file_details)} files indexed with {total_documents} total chunks")
    print(f"✅ German search functionality working")
else:
    print(f"⚠️ ISSUES DETECTED:")
    if missing_files:
        print(f"   - {len(missing_files)} expected files missing")
    if not all_files_complete:
        print(f"   - Some files have incorrect chunk counts")
    if not german_found:
        print(f"   - German search not working")
print(f"{'='*60}")

🔍 Final verification of the indexed content...
📊 INDEX SUMMARY:
   Total documents indexed: 11
   Unique files: 5

📋 DETAILED FILE BREAKDOWN:
   ✅ Baier_2015_Migration und KriminalitÃ¤t_converted.md
      Type: Markdown Document
      Expected chunks: 3
      Indexed chunks: 3

   ✅ City of Munich_2017_Flugblatt_Kriminalitaet_converted.md
      Type: Markdown Document
      Expected chunks: 1
      Indexed chunks: 1

   ✅ Goeckenjan_2019_FluchtalsSicherheitsproblem_converted.md
      Type: Markdown Document
      Expected chunks: 3
      Indexed chunks: 3

   ✅ Statista_2025_Ausländerkriminalität in Deutschland.pdf
      Type: PDF Document
      Expected chunks: 3
      Indexed chunks: 3

   ✅ merged_statistic_id558217.csv
      Type: CSV Data Table
      Expected chunks: 1
      Indexed chunks: 1

❌ MISSING FILES:
   🚫 Statista_2025_Ausländerkriminalität in Deutschland.pdf

🧪 TESTING GERMAN SEARCH:
   📄 Statista_2025_Ausländerkriminalität in Deutschland.pdf: Statista_2025_Ausländ